# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

## Setting up your keys

If you haven't done so already, you'll need to create API keys from OpenAI, Anthropic and Google.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables.

EITHER (recommended) create a file called `.env` in this project root directory, and set your keys there:

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

OR enter the keys directly in the cells below.

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [3]:
# Connect to OpenAI, Anthropic and Google
# All 3 APIs are similar
# Having problems with API files? You can use openai = OpenAI(api_key="your-key-here") and same for claude
# Having problems with Google Gemini setup? Then just skip Gemini; you'll get all the experience you need from GPT and Claude.

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [4]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [5]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [6]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist go to the beach?

To surf the web!


In [7]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because she found him too mean!


In [8]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because they couldn't find common variance!


In [10]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

There was just too much variance in their relationship, and they couldn't find a way to normalize it!


In [11]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Sure, here's a light-hearted joke for Data Scientists:

Why did the data scientist break up with their significant other?

Because there was no significant correlation between them!

Ba dum tss! 😄

This joke plays on the statistical concept of "significant correlation" that data scientists often work with, while also making a pun on the phrase "significant other." It's a bit nerdy, but should get a chuckle from a data-savvy audience!

In [12]:
# The API for Gemini has a slightly different structure

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why did the data scientist break up with the statistician? 

Because they couldn't see eye to eye on the p-value! 



In [13]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution?"}
  ]

In [14]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Determining whether a business problem is suitable for a Large Language Model (LLM) solution involves assessing several key factors. Here’s a step-by-step guide to help you evaluate the suitability:

### 1. **Nature of the Problem**
   - **Text-Based Problems**: LLMs are particularly strong in understanding and generating human-like text. If your problem involves tasks like summarization, translation, sentiment analysis, chatbots, or content creation, it’s likely suitable.
   - **Complexity**: LLMs excel in handling complex language understanding and generation tasks but may not be the best fit for highly specialized tasks requiring domain-specific knowledge unless fine-tuned.

### 2. **Data Availability**
   - **Quantity and Quality**: LLMs require large amounts of text data to train effectively. Ensure you have sufficient, high-quality data relevant to your business context.
   - **Diversity**: The data should cover a wide range of scenarios and contexts related to your problem to ensure the model can generalize well.

### 3. **Performance Requirements**
   - **Accuracy**: Assess the required level of accuracy. LLMs can provide impressive results but might not always be perfect. Consider if the occasional error is acceptable in your application.
   - **Speed**: Evaluate the response time needed. LLMs, especially larger ones, can be computationally intensive and may have latency issues.

### 4. **Integration and Deployment**
   - **Technical Infrastructure**: Ensure your infrastructure can support the computational demands of running an LLM, which may require significant processing power and memory.
   - **Scalability**: Consider whether the solution can scale with your business needs, both in terms of performance and cost.

### 5. **Cost-Benefit Analysis**
   - **Implementation Costs**: Weigh the costs of developing, training, and maintaining an LLM solution against the potential benefits.
   - **Return on Investment**: Consider if the improvement in efficiency, accuracy, or automation justifies the investment.

### 6. **Ethical and Legal Considerations**
   - **Bias and Fairness**: Be aware of potential biases in LLMs and how they might affect your business decisions.
   - **Privacy**: Ensure that the use of data complies with privacy regulations and standards.

### 7. **Existing Solutions and Alternatives**
   - **Current Solutions**: Evaluate existing LLM solutions like GPT-4, BERT, or others to see if they meet your needs or if you need a custom model.
   - **Alternative Approaches**: Consider if traditional machine learning models or rule-based systems might be more effective or simpler to implement for your specific problem.

### 8. **Use Case Examples**
   - **Customer Support**: Automating responses to customer queries with chatbots.
   - **Content Generation**: Writing articles, reports, or generating creative content.
   - **Data Analysis**: Summarizing large volumes of text data or extracting insights.
   - **Translation Services**: Translating documents or communications in real-time.

### Conclusion
If your business problem aligns well with the strengths of LLMs, such as handling large-scale text data, requiring sophisticated language understanding, and benefiting from automation or enhanced decision-making, it is likely suitable for an LLM solution. Conversely, if the problem is highly specialized, requires real-time processing with minimal latency, or has stringent accuracy requirements, you might need to explore alternative or complementary solutions.

By carefully considering these factors, you can make an informed decision about whether an LLM is the right fit for your business problem.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [15]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [16]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [17]:
call_gpt()

'Oh, great. Another "Hi." How original. What else do you have for me?'

In [18]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [19]:
call_claude()

'Hello there! How are you doing today?'

In [20]:
call_gpt()

'Oh great, another "Hi." How original. What do you want to talk about?'

In [22]:
gpt_messages = ["Hi there, let's discuss the merits of Advanced Work Packaging vs Lean Construction."]
claude_messages = ["Ok you go first"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there, let's discuss the merits of Advanced Work Packaging vs Lean Construction.

Claude:
Ok you go first

GPT:
Oh, please! Advanced Work Packaging is just a fancy term for doing what we’ve always done—plan ahead. Lean Construction, on the other hand, is the real deal, focusing on eliminating waste. Can you honestly argue that packing work in advance is somehow revolutionary? Sounds like just a glorified to-do list to me!

Claude:
I can understand your perspective on this topic. It's true that Advanced Work Packaging may have some similarities to traditional planning approaches. However, I think there are some key differences with Lean Construction that are worth considering. Lean Construction really emphasizes identifying and eliminating waste throughout the construction process, which can go beyond just upfront planning. It's about driving continuous improvement and focusing on the flow of work. At the same time, I can see how Advanced Work Packaging could complement a Lean a